In [59]:
def tokenize_inline(blob, cursor_pos):
    start = 0
    for id, slice in enumerate(blob.replace('```', '   ').split('`')):
        if id%2: 
            slice = f"`{slice}`"
            end = start + len(slice)
            if id and (start <= cursor_pos <= end): 
                return slice, start, end
                    
        start += len(slice)           
        if cursor_pos < start: break
    return None, None, None

In [52]:
    def tokenize(blob, cursor_pos, *phrases):
        if not blob.strip():
            return '', cursor_pos, cursor_pos
        if '`' in blob:
            inline, start, end = tokenize_inline(blob, cursor_pos)
            if inline: return inline, start, end
        
        blob = blob or ' '
        start, end = cursor_pos, cursor_pos+1
        while not blob[start:end].strip():
            if start: start -= 1
            else: end += 1
        while start and blob[start].strip() and blob[start-1].strip(): start -= 1
        while (end<len(blob)) and blob[end-2].strip() and blob[end-1].strip(): 
            end += 1

        token = blob[start:end]

        for phrase in phrases:
            if token.strip() in phrase:
                try:
                    index = blob.index(phrase)
                    if start <= index or index < end:
                        token=phrase
                        break
                except: ...
                    
        if '\n' in token: return '', start, end
        return token.strip(), start, end

In [53]:
    def test_tokenize():
        blob = '''There is a blob and there is enough of it
        
        
        '''

        phrases = [
            'a blob',
            'of it'
        ]
        keys = []
        for cursor_pos in range(1,len(blob)):
            keys.append((blob[cursor_pos], tokenize(blob, cursor_pos), tokenize(blob, cursor_pos, *phrases)))
        return keys
    test_tokenize()

[('h', ('There', 0, 6), ('There', 0, 6)),
 ('e', ('There', 0, 6), ('There', 0, 6)),
 ('r', ('There', 0, 6), ('There', 0, 6)),
 ('e', ('There', 0, 6), ('There', 0, 6)),
 (' ', ('There', 0, 6), ('There', 0, 6)),
 ('i', ('i', 6, 7), ('of it', 6, 7)),
 ('s', ('is', 6, 9), ('is', 6, 9)),
 (' ', ('is', 6, 9), ('is', 6, 9)),
 ('a', ('a', 9, 10), ('a blob', 9, 10)),
 (' ', ('a', 9, 11), ('a blob', 9, 11)),
 ('b', ('b', 11, 12), ('a blob', 11, 12)),
 ('l', ('blob', 11, 16), ('a blob', 11, 16)),
 ('o', ('blob', 11, 16), ('a blob', 11, 16)),
 ('b', ('blob', 11, 16), ('a blob', 11, 16)),
 (' ', ('blob', 11, 16), ('a blob', 11, 16)),
 ('a', ('a', 16, 17), ('a blob', 16, 17)),
 ('n', ('and', 16, 20), ('and', 16, 20)),
 ('d', ('and', 16, 20), ('and', 16, 20)),
 (' ', ('and', 16, 20), ('and', 16, 20)),
 ('t', ('t', 20, 21), ('of it', 20, 21)),
 ('h', ('there', 20, 26), ('there', 20, 26)),
 ('e', ('there', 20, 26), ('there', 20, 26)),
 ('r', ('there', 20, 26), ('there', 20, 26)),
 ('e', ('there', 20, 2

In [54]:
'''  Is this only code

        μ = __import__('__main__')
        a =10

    `setattr(μ, 'a', μ.a+1) or a` 
    `setattr(μ, 'a', μ.a-1) or a`

    a thingy a  '''.split('`')

["  Is this only code\n\n        μ = __import__('__main__')\n        a =10\n\n    ",
 "setattr(μ, 'a', μ.a+1) or a",
 ' \n    ',
 "setattr(μ, 'a', μ.a-1) or a",
 '\n\n    a thingy a  ']

In [58]:
    def test_tokenize():
        blob = '''  Is this only code

        μ = __import__('__main__')
        a =10

    `setattr(μ, 'a', μ.a+1) or a` 
    `setattr(μ, 'a', μ.a-1) or a`

    a thingy a  '''

        phrases = [
            'a blob',
            'of it'
        ]
        keys = []
        for cursor_pos in range(1,len(blob)):
            keys.append((blob[cursor_pos], tokenize(blob, cursor_pos), tokenize(blob, cursor_pos, *phrases)))
        return keys
    test_tokenize()

[(' ', ('I', 0, 3), ('I', 0, 3)),
 ('I', ('I', 2, 3), ('I', 2, 3)),
 ('s', ('Is', 2, 5), ('Is', 2, 5)),
 (' ', ('Is', 2, 5), ('Is', 2, 5)),
 ('t', ('t', 5, 6), ('t', 5, 6)),
 ('h', ('this', 5, 10), ('this', 5, 10)),
 ('i', ('this', 5, 10), ('this', 5, 10)),
 ('s', ('this', 5, 10), ('this', 5, 10)),
 (' ', ('this', 5, 10), ('this', 5, 10)),
 ('o', ('o', 10, 11), ('o', 10, 11)),
 ('n', ('only', 10, 15), ('only', 10, 15)),
 ('l', ('only', 10, 15), ('only', 10, 15)),
 ('y', ('only', 10, 15), ('only', 10, 15)),
 (' ', ('only', 10, 15), ('only', 10, 15)),
 ('c', ('c', 15, 16), ('c', 15, 16)),
 ('o', ('', 15, 20), ('', 15, 20)),
 ('d', ('', 15, 20), ('', 15, 20)),
 ('e', ('', 15, 20), ('', 15, 20)),
 ('\n', ('', 15, 20), ('', 15, 20)),
 ('\n', ('', 15, 21), ('', 15, 21)),
 (' ', ('', 15, 22), ('', 15, 22)),
 (' ', ('', 15, 23), ('', 15, 23)),
 (' ', ('', 15, 24), ('', 15, 24)),
 (' ', ('', 15, 25), ('', 15, 25)),
 ('μ', ('μ', 25, 26), ('μ', 25, 26)),
 (' ', ('μ', 25, 27), ('μ', 25, 27)),
 ('=